In [0]:
!pip install pydrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1-U4nlAp57FOzQL3GoB4CETCbjbluGIKP'})
download.GetContentFile('/content/spatial.7z')


!7z e /content/spatial.7z

!mkdir /content/scenes/
!mkdir /content/scenes/coast/
!mkdir /content/scenes/forest/
!mkdir /content/scenes/highway/
!mkdir /content/nst/
!mkdir /content/nst/styledata/
# !mkdir /content/scenes/insidecity/
#!mkdir /content/scenes/mountain/
#!mkdir /content/scenes/opencountry/
# !mkdir /content/scenes/street/
# !mkdir /content/scenes/tallbuilding/

!mv /content/coast*.jpg /content/scenes/coast
!mv /content/forest*.jpg /content/scenes/forest
!mv /content/highway*.jpg /content/scenes/highway
!mv /content/10power*.jpg /content/nst/styledata
# !mv /content/insidecity*.jpg /content/scenes/insidecity
#!mv /content/mountain*.jpg /content/scenes/mountain
#!mv /content/opencountry*.jpg /content/scenes/opencountry
# !mv /content/street*.jpg /content/scenes/street
# !mv /content/tallbuilding*.jpg /content/scenes/tallbuilding
!rm /content/*.jpg


In [0]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from PIL import Image
from imutils import paths
import numpy as np
import argparse
import os

def extract_color_stats(image):
	# split the input image into its respective RGB color channels
	# and then create a feature vector with 6 values: the mean and
	# standard deviation for each of the 3 channels, respectively
	(R, G, B) = image.convert('RGB').split()
	features = [np.mean(R), np.mean(G), np.mean(B), np.std(R),
		np.std(G), np.std(B)]

	# return our set of features
	return features


models = {
    "naive_bayes": GaussianNB(),
    "decision_tree": DecisionTreeClassifier(),
    "svm": SVC(kernel="linear"),
	"knn": KNeighborsClassifier(n_neighbors=1),
	"logit": LogisticRegression(solver="lbfgs", multi_class="auto"),
	"random_forest": RandomForestClassifier(n_estimators=100),
	"mlp": MLPClassifier()
}


print("[INFO] extracting image features...")
imagePaths = paths.list_images('/content/scenes/')
nstPath = paths.list_images('/content/nst/')
data = []
labels = []
nst_data = []
nst_labels =[]


# loop over our input images
for imagePath in imagePaths:
	# load the input image from disk, compute color channel
	# statistics, and then update our data list
  image = Image.open(imagePath)
  features = extract_color_stats(image)
  data.append(features)
	# extract the class label from the file path and update the
	# labels list
  label = imagePath.split(os.path.sep)[-2]
  labels.append(label)
  
  
for imagePath2 in nstPath:
  image = Image.open(imagePath2)
  features = extract_color_stats(image)
  nst_data.append(features)
  label = imagePath2.split('/')[-1]  
  nst_labels.append(label)
  
  
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels)
#nst_labels = le.fit_transform(nst_labels)



(trainX, testX, trainY, testY) = train_test_split(data,labels,
	test_size=0.25)


[INFO] extracting image features...


In [0]:
import math
i=0
for name, mod in models.items():
  model = mod
  model.fit(trainX, trainY)


  #make predictions on our data and show a classification report
  print("[INFO] evaluating for",name,"...")
  predictions = model.predict(testX)
  print(classification_report(testY, predictions, target_names=le.classes_))
  
  print("[INFO] evaluating styled images using ",name," algorithm")
  nst_predictions = np.array(le.inverse_transform(model.predict(nst_data)))
  print(nst_labels,"\t",nst_predictions)
 

[INFO] evaluating for naive_bayes ...
              precision    recall  f1-score   support

       coast       0.76      0.37      0.50        68
      forest       0.77      0.88      0.82        88
     highway       0.60      0.83      0.70        60

    accuracy                           0.70       216
   macro avg       0.71      0.69      0.67       216
weighted avg       0.72      0.70      0.68       216

[INFO] evaluating styled images using  naive_bayes  algorithm
['10power8.jpg', '10power5.jpg', '10power7.jpg', '10power-1.jpg', '10power6.jpg'] 	 ['highway' 'highway' 'highway' 'highway' 'highway']
[INFO] evaluating for decision_tree ...
              precision    recall  f1-score   support

       coast       0.70      0.62      0.66        68
      forest       0.85      0.85      0.85        88
     highway       0.57      0.65      0.61        60

    accuracy                           0.72       216
   macro avg       0.71      0.71      0.71       216
weighted avg     

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[INFO] evaluating for random_forest ...
              precision    recall  f1-score   support

       coast       0.82      0.75      0.78        68
      forest       0.88      0.93      0.91        88
     highway       0.80      0.82      0.81        60

    accuracy                           0.84       216
   macro avg       0.84      0.83      0.83       216
weighted avg       0.84      0.84      0.84       216

[INFO] evaluating styled images using  random_forest  algorithm
['10power8.jpg', '10power5.jpg', '10power7.jpg', '10power-1.jpg', '10power6.jpg'] 	 ['forest' 'coast' 'highway' 'coast' 'highway']
[INFO] evaluating for mlp ...
              precision    recall  f1-score   support

       coast       0.83      0.78      0.80        68
      forest       0.93      0.91      0.92        88
     highway       0.76      0.83      0.79        60

    accuracy                           0.85       216
   macro avg       0.84      0.84      0.84       216
weighted avg       0.85     